In [45]:
#This program loads the update client message type into its related transaction detail table inside of the blockchain db
import pandas as pd
import json
import psycopg2
import logging

DB_NAME = 'blockchain'
DB_USER = 'postgres'
DB_PASSWORD = 'auth2020'
DB_HOST = 'localhost'
DB_PORT = '5432'


try:
    # Create a PostgreSQL connection
    conn = psycopg2.connect(
        dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT
    )
    cur = conn.cursor()
    print("Connection established.")  # Log successful connection (you can modify this)

    # Configure logging to write to a log file
    logging.basicConfig(filename='log.txt', level=logging.INFO)
    logging.info("Connection established with Database.")  # Log the same message to the file


    # Additional code for database operations goes here...

except Exception as err:
    # Handle exceptions
    print("Unable to connect to database.")  # Log error (you can modify this)
    print(err)  # Print the specific error message

    # Optionally, write the error message to a file (error.txt)
    with open('error.txt', 'a') as error_file:
        error_file.write(str(err) + '\n')

Connection established.


In [46]:

def get_most_recent_transaction_claim_id(conn):
    try:
        with conn.cursor() as cursor:
            # Query to retrieve the most recent transaction_exec_id
            get_transaction_claim_id_query = """
                SELECT transaction_claim_id
                FROM transaction_header_claim_delegate
                ORDER BY transaction_claim_id DESC
                LIMIT 1;
            """
            cursor.execute(get_transaction_claim_id_query)
            result = cursor.fetchone()
            if result:
                return result[0]
            else:
                print("No transaction_claim_id found.")
                return None
    except psycopg2.Error as e:
        print(f"Error retrieving transaction_id: {e}")
        return None

In [47]:

def main():
    block = '5929489'
     # Change this number if you want to load a different file
    file_path = f'C:/Users/lmhmo/Indexer_Project/indexer_project/Decrypted_files/{block}.json'

    try:
        # Check if the file exists
        if os.path.exists(file_path):
            print(f"Found JSON file: {file_path}")

            # Load JSON data from the file
            with open(file_path, 'r') as json_file:
                json_data = json.load(json_file)

                # Connect to the database
                conn = connect_to_database()

                # Get the most recent transaction_exec_id
                transaction_claim_id = get_most_recent_transaction_claim_id(conn)

                if transaction_claim_id:
                    # Initialize an empty list to store insert queries
                    insert_queries = []

                    # Iterate through the messages
                    for message in json_data['tx']['body']['messages']:
                        # Extract data from the given JSON structure
                        delegator_address_value = message["delegator_address"]
                        validator_address_value = message["validator_address"]
                        denom_value = message["denom"]

                        # Construct the insert query for the current message
                        msg_insert_query = f"""
                            INSERT INTO transaction_detail_msg_claim_delegation_rewards (transaction_claim_id, delegator_address, validator_address, denom)
                            VALUES ('{transaction_claim_id}','{delegator_address_value}', '{validator_address_value}', '{denom_value}');
                        """
                        insert_queries.append(msg_insert_query)

                    # Execute the insert queries
                    with conn.cursor() as cursor:
                        for query in insert_queries:
                            cursor.execute(query)
                        conn.commit()  # Commit the transaction

                    print("Data inserted successfully!")
                else:
                    print("No transaction_exec_id found. Data not inserted.")

            # Close the database connection
            conn.close()
        else:
            print(f"JSON file not found: {file_path}")
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()



Found JSON file: C:/Users/lmhmo/Indexer_Project/indexer_project/Decrypted_files/5929489.json
Connected to the database successfully!
Data inserted successfully!
